In [2]:
import numpy as np
import os
import re
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
import opensmile

In [3]:
## Variables Defined
path=r"D:\PRML"
max_count=4000  ## No. of examples to use for training

train_path=os.path.join(path,"train","train")
test_path=os.path.join(path,"test","test")

In [4]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,   ## Geneva Minimalistic Acoustic Parameter Set
    feature_level=opensmile.FeatureLevel.Functionals,
)

## Training

In [5]:
true_values=[]
count=0
count_male=0
count_female=0
features=[]
flag=0
for folder in os.listdir(train_path):
    folder_path=os.path.join(train_path,folder)
    for file in os.listdir(folder_path):
        if(count==max_count):
            break
        if(re.search("_f_",file)):
            if(count_female>=max_count/2):
                continue
            count_female+=1
            if(count_female==max_count/2):
                print("Female for "+str(folder)+" done")

        elif(re.search("_m_",file)):
            if(count_male>=max_count/2):
                continue
            count_male+=1
            if(count_male==max_count/2):
                print("Male for "+str(folder)+" done")
        
        file_path=os.path.join(folder_path,file)
        y=smile.process_file(file_path)
        file_features=np.asarray(y)
        if flag==0:
            features=file_features
            flag=-1
        else:
            features=np.concatenate((features,file_features),axis=0)
        if(re.match("de",file)):
            true_values.append(0)
        elif(re.match("en",file)):
            true_values.append(1)
        elif(re.match("es",file)):
            true_values.append(2)
        count+=1
    
    count=0
    count_female=0
    count_male=0
    print(str(folder)+" Done")
true_values=np.array(true_values)   


print("Feature Extraction Done")
        


Female for English done
Male for English done
English Done
Female for German done


In [ ]:
print(features.shape)
print(true_values.shape)

## Testing

In [ ]:
true_values_test=[]
test_features=[]
count=0
flag=0
for folder in os.listdir(test_path):
    folder_path=os.path.join(test_path,folder)
    for file in os.listdir(folder_path):
        file_path=os.path.join(folder_path,file)
        y=smile.process_file(file_path)
        file_features=np.asarray(y)
        if flag==0:
            test_features=file_features
            flag=-1
        else:
            test_features=np.concatenate((test_features,file_features),axis=0)
        if(re.match("de",file)):
            true_values_test.append(0)
        elif(re.match("en",file)):
            true_values_test.append(1)
        elif(re.match("es",file)):
            true_values_test.append(2)
        count+=1
true_values_test=np.array(true_values_test)   

print(count)
print("Feature Extraction Done")
        


In [ ]:
res=[24,38,58,60,68,70,76,78] 
 
# 24= mfcc sma3 mean
# 38= log RelF0-H1-A3_sma3_nz_amean
# 58= AlphaRatioV_sma3_nz_amean
# 60= hammerberg index_sma3_nz_amean
# 68= mfcc1V sma3nz amean
# 70= mfcc2V sma3nz amean
# 76= alphaRatioUV_sma3_nz_amean
# 78= slopeUV0-500 sma3nz amean 

res.T
features=features[:,res]
test_features=test_features[:,res]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler


# Linear


In [ ]:
from sklearn.svm import SVC

pipe4 = Pipeline([ ('std', StandardScaler()), ('SVM', SVC(kernel='linear',probability=True))])
pipe4.fit(features, true_values)
score=accuracy_score(true_values_test, pipe4.predict(test_features))
print(score)
